In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime as dt

In [5]:
QVI_data = pd.read_csv("QVI_data.csv")

In [6]:
QVI_data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [7]:
QVI_data.describe()

,LYLTY_CARD_NBR,STORE_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES,PACK_SIZE
count,2.648340e+05,264834.000000,2.648340e+05,264834.000000,264834.000000,264834.000000,264834.000000
mean,1.355488e+05,135.079423,1.351576e+05,56.583554,1.905813,7.299346,182.425512
std,8.057990e+04,76.784063,7.813292e+04,32.826444,0.343436,2.527241,64.325148
min,1.000000e+03,1.000000,1.000000e+00,1.000000,1.000000,1.500000,70.000000
25%,7.002100e+04,70.000000,6.760050e+04,28.000000,2.000000,5.400000,150.000000
50%,1.303570e+05,130.000000,1.351365e+05,56.000000,2.000000,7.400000,170.000000
75%,2.030940e+05,203.000000,2.026998e+05,85.000000,2.000000,9.200000,175.000000
max,2.373711e+06,272.000000,2.415841e+06,114.000000,5.000000,29.500000,380.000000


In [8]:
QVI_data.isnull().sum()

LYLTY_CARD_NBR      0
DATE                0
STORE_NBR           0
TXN_ID              0
PROD_NBR            0
PROD_NAME           0
PROD_QTY            0
TOT_SALES           0
PACK_SIZE           0
BRAND               0
LIFESTAGE           0
PREMIUM_CUSTOMER    0
dtype: int64

In [10]:
total_sales_revenue = sum(QVI_data["TOT_SALES"])

In [11]:
total_customers = QVI_data["LYLTY_CARD_NBR"].nunique()

In [29]:
QVI_data['month_id'] = QVI_data["DATE"].apply(lambda x:x[:7])

In [44]:
QVI_data

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,month_id
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018-10
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018-09
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,2019-03
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,2019-03
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream,2018-12
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium,2018-10
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget,2018-10
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget,2018-10


In [134]:
Measure_over_time = QVI_data[["TOT_SALES","PROD_QTY","month_id","STORE_NBR"]]
Measure_over_time = Measure_over_time.groupby(by = ["month_id","STORE_NBR"]).sum().reset_index()

Customers = QVI_data[["LYLTY_CARD_NBR","month_id","STORE_NBR"]]
Customers = Customers.groupby(by = ["month_id","STORE_NBR"]).nunique()
Customers = Customers["LYLTY_CARD_NBR"].reset_index()

Measure_over_time = Measure_over_time.merge(Customers)
Measure_over_time["Avg_Sales"] = Measure_over_time.apply(lambda x: x.TOT_SALES/x.LYLTY_CARD_NBR,axis = 1)
Measure_over_time["Chips_per_person"] = Measure_over_time.apply(lambda x: x.PROD_QTY/x.LYLTY_CARD_NBR,axis = 1)

In [148]:
Measure_over_time[:267]

,month_id,STORE_NBR,TOT_SALES,PROD_QTY,LYLTY_CARD_NBR,Avg_Sales,Chips_per_person
0,2018-07,1,206.9,62,49,4.222449,1.265306
1,2018-07,2,150.8,46,39,3.866667,1.179487
2,2018-07,3,1205.7,271,112,10.765179,2.419643
3,2018-07,4,1399.9,318,128,10.936719,2.484375
4,2018-07,5,812.0,240,93,8.731183,2.580645
...,...,...,...,...,...,...,...
262,2018-07,269,982.0,278,103,9.533981,2.699029
263,2018-07,270,962.8,278,96,10.029167,2.895833
264,2018-07,271,956.6,258,96,9.964583,2.687500
265,2018-07,272,433.1,98,48,9.022917,2.041667


In [138]:
import plotly.express as px

In [147]:
fig = px.bar(Measure_over_time,Measure_over_time['STORE_NBR'],Measure_over_time['LYLTY_CARD_NBR'])
fig.update_layout(title='Customers per store ',title_x=0.5)

In [145]:
fig = px.bar(Measure_over_time,Measure_over_time['STORE_NBR'],Measure_over_time['Avg_Sales'])
fig.update_layout(title='Store wise sales',title_x=0.5)

In [164]:
data_corelation = Measure_over_time[["STORE_NBR","LYLTY_CARD_NBR","Avg_Sales"]]

In [168]:
data_corelation = data_corelation.groupby("STORE_NBR").sum()/12

In [172]:
data_corelation = data_corelation.reset_index()

In [197]:
data_corelation[(data_corelation["STORE_NBR"] == 77)|( data_corelation["STORE_NBR"] == 86)|( data_corelation["STORE_NBR"] == 88)] 

,STORE_NBR,LYLTY_CARD_NBR,Avg_Sales
76,77,44.750000,5.672919
85,86,102.166667,8.673244
87,88,126.250000,10.787047


In [200]:
data_corelation[(data_corelation["Avg_Sales"] >5.45)&(data_corelation["Avg_Sales"] <5.9)&(data_corelation["LYLTY_CARD_NBR"] > 43)&(data_corelation["LYLTY_CARD_NBR"] < 46)]

,STORE_NBR,LYLTY_CARD_NBR,Avg_Sales
37,38,45.25,5.793705
49,50,45.50,5.512779
76,77,44.75,5.672919


In [201]:
data_corelation[(data_corelation["Avg_Sales"] >8)&(data_corelation["Avg_Sales"] <9.25)&(data_corelation["LYLTY_CARD_NBR"] > 101)&(data_corelation["LYLTY_CARD_NBR"] < 103)]

,STORE_NBR,LYLTY_CARD_NBR,Avg_Sales
32,33,102.833333,9.043340
56,57,102.083333,8.613117
85,86,102.166667,8.673244
179,180,101.500000,8.924432
246,247,101.500000,8.738250
269,270,102.833333,9.170371


In [202]:
data_corelation[(data_corelation["Avg_Sales"] >10)&(data_corelation["Avg_Sales"] <11.25)&(data_corelation["LYLTY_CARD_NBR"] > 125)&(data_corelation["LYLTY_CARD_NBR"] < 127)]

,STORE_NBR,LYLTY_CARD_NBR,Avg_Sales
39,40,125.25,10.364175
87,88,126.25,10.787047


**Seems like stores 50,57 & 40 are good candidates for control stores of trial stores 77, 86 & 88  respectively** 